# 用Python读取nc文件的入门级操作

In [1]:
import numpy as np
import netCDF4 as nc

将文件路径存入变量**file_path**中

In [2]:
file_path = './air.mon.mean.nc'

使用netCDF4的Dataset方法读取文件，并把netCDF4文件对象赋值给变量**file_obj**

In [3]:
file_obj = nc.Dataset(file_path)

此时的**file_obj**只是一个对象，可以直接执行以查看它的概述

## 属性的查看和数据的读取

In [4]:
file_obj

<class 'netCDF4._netCDF4.Dataset'>
root group (NETCDF4_CLASSIC data model, file format HDF5):
    description: Data from NCEP initialized reanalysis (4x/day).  These are the 0.9950 sigma level values
    platform: Model
    Conventions: COARDS
    NCO: 20121012
    history: Thu May  4 20:11:16 2000: ncrcat -d time,0,623 /Datasets/ncep.reanalysis.derived/surface/air.mon.mean.nc air.mon.mean.nc
Thu May  4 18:11:50 2000: ncrcat -d time,0,622 /Datasets/ncep.reanalysis.derived/surface/air.mon.mean.nc ./surface/air.mon.mean.nc
Mon Jul  5 23:47:18 1999: ncrcat ./air.mon.mean.nc /Datasets/ncep.reanalysis.derived/surface/air.mon.mean.nc /dm/dmwork/nmc.rean.ingest/combinedMMs/surface/air.mon.mean.nc
/home/hoop/crdc/cpreanjuke2farm/cpreanjuke2farm Mon Oct 23 21:04:20 1995 from air.sfc.gauss.85.nc
created 95/03/13 by Hoop (netCDF2.3)
Converted to chunked, deflated non-packed NetCDF4 2014/09
    title: monthly mean air.sig995 from the NCEP Reanalysis
    References: http://www.esrl.noaa.gov/psd/dat

可以看到，这是一个叫做**netCDF4._netCDF4.Dataset**的类对象   


上面的显示内容太混乱，如果我们想要快速获取该对象中存储的变量名，可以执行下面这条语句

In [5]:
file_obj.variables.keys()  # 在Python2.7版本中输出结果为列表

odict_keys(['lat', 'lon', 'time', 'air'])

这样就显示了file_obj对象里存储的4个变量的变量名，即纬度(lat)，经度(lon)，时间(time)   
而对于最后一个air是什么意思？我们可能并不好确定   
那么我们就可以把它读取出来

In [6]:
air = file_obj.variables['air']

通过直接运行变量名，我们可以查看它的属性信息

In [7]:
air

<class 'netCDF4._netCDF4.Variable'>
float32 air(time, lat, lon)
    long_name: Monthly Mean Air Temperature at sigma level 0.995
    valid_range: [-2000.  2000.]
    units: degC
    add_offset: 0.0
    scale_factor: 1.0
    missing_value: -9.96921e+36
    precision: 1
    least_significant_digit: 0
    var_desc: Air Temperature
    level_desc: Surface
    statistic: Mean
    parent_stat: Individual Obs
    dataset: NCEP Reanalysis Derived Products
    actual_range: [-73.78000641  42.14595032]
unlimited dimensions: time
current shape = (848, 73, 144)
filling on, default _FillValue of 9.969209968386869e+36 used

通常情况下nc文件中每个变量都会封装其属性，一般会包括全名（long_name)，单位（units）等等   
可以看到，air变量的全名(long_name)是Monthly Mean Air Temperature at sigma level 0.995，也就是气温  

除了全名、单位等常规信息，以下几条信息你需要特别注意： 

1、变量的维度信息：   
属性信息的第二行显示了该变量的维度组成，即air(time, lat, lon)。这表示该变量是三维数组，其维度顺序是时间、纬度、经度。  

2、缺省值信息：   
通常情况下，nc文件中所存储的三维空间场数据会有一些空间点缺少数据，但是这些地方不能空着，就会被缺省值取代，这个缺省值不会是0，一般都是一个非常大的数字或者非常小的数字，这个数字在正常数据中是不可能出现的，例如本例中的missing_value: -9.96921e+36，在对数据的处理过程中要考虑和重视缺省值，否则你最后可能会处理处虚假的结果，这个在以后的文章中再详细说明

3、解包参数：   
nc文件有个特点，就是有时候数据占位比较多，会比较费空间。因此人们在对nc文件进行封装处理的时候，通常会有一个**数据打包**的过程，这个过程会使用一种算法改变数据的值，然后使用更节省空间数据类型来进行存储。在实际使用的时候再通过**解包**算法将其还原为真实值。   
 
因此，如果你遇到一个nc文件，读出来的数据很奇怪（很整齐的整数、不合理的数据范围），那么你就要考虑一下是不是需要进行解包。

解包算法是这样的：真实值 = 打包值 * **scale_factor** + **add_offset**  

现在我们要开始读取air变量的值了，读取过程非常简单

In [8]:
air_arr = air[:]

可以查看一下air_arr的形状

In [9]:
air_arr.shape

(848, 73, 144)

这是一个 848 x 73 x 144 的数字阵列，由于矩阵太过庞大，下面只显示其一个时次的数据内容

In [10]:
air_arr[0]

array([[-34.92677307, -34.92677307, -34.92677307, ..., -34.92677307,
        -34.92677307, -34.92677307],
       [-35.13935089, -35.129673  , -35.12741852, ..., -35.18870544,
        -35.17000198, -35.14934921],
       [-34.35257339, -34.04225922, -33.76870728, ..., -35.33386612,
        -35.00290298, -34.67128754],
       ..., 
       [-16.52515602, -16.40450859, -16.284832  , ..., -16.79515457,
        -16.73773575, -16.64354324],
       [-16.19031334, -16.20224762, -16.21677017, ..., -16.13257408,
        -16.16192818, -16.17837715],
       [-17.69773293, -17.69773293, -17.69773293, ..., -17.69773293,
        -17.69773293, -17.69773293]], dtype=float32)

经度和维度的读取

In [11]:
lon = file_obj.variables['lon'][:]
lat = file_obj.variables['lat'][:]

In [12]:
lon

array([   0. ,    2.5,    5. ,    7.5,   10. ,   12.5,   15. ,   17.5,
         20. ,   22.5,   25. ,   27.5,   30. ,   32.5,   35. ,   37.5,
         40. ,   42.5,   45. ,   47.5,   50. ,   52.5,   55. ,   57.5,
         60. ,   62.5,   65. ,   67.5,   70. ,   72.5,   75. ,   77.5,
         80. ,   82.5,   85. ,   87.5,   90. ,   92.5,   95. ,   97.5,
        100. ,  102.5,  105. ,  107.5,  110. ,  112.5,  115. ,  117.5,
        120. ,  122.5,  125. ,  127.5,  130. ,  132.5,  135. ,  137.5,
        140. ,  142.5,  145. ,  147.5,  150. ,  152.5,  155. ,  157.5,
        160. ,  162.5,  165. ,  167.5,  170. ,  172.5,  175. ,  177.5,
        180. ,  182.5,  185. ,  187.5,  190. ,  192.5,  195. ,  197.5,
        200. ,  202.5,  205. ,  207.5,  210. ,  212.5,  215. ,  217.5,
        220. ,  222.5,  225. ,  227.5,  230. ,  232.5,  235. ,  237.5,
        240. ,  242.5,  245. ,  247.5,  250. ,  252.5,  255. ,  257.5,
        260. ,  262.5,  265. ,  267.5,  270. ,  272.5,  275. ,  277.5,
      

In [13]:
lat

array([ 90. ,  87.5,  85. ,  82.5,  80. ,  77.5,  75. ,  72.5,  70. ,
        67.5,  65. ,  62.5,  60. ,  57.5,  55. ,  52.5,  50. ,  47.5,
        45. ,  42.5,  40. ,  37.5,  35. ,  32.5,  30. ,  27.5,  25. ,
        22.5,  20. ,  17.5,  15. ,  12.5,  10. ,   7.5,   5. ,   2.5,
         0. ,  -2.5,  -5. ,  -7.5, -10. , -12.5, -15. , -17.5, -20. ,
       -22.5, -25. , -27.5, -30. , -32.5, -35. , -37.5, -40. , -42.5,
       -45. , -47.5, -50. , -52.5, -55. , -57.5, -60. , -62.5, -65. ,
       -67.5, -70. , -72.5, -75. , -77.5, -80. , -82.5, -85. , -87.5, -90. ], dtype=float32)

## 时间的格式转换

下面我们来看一下nc文件中时间维的数据存储方式是什么样的

In [14]:
time = file_obj.variables['time']
time

<class 'netCDF4._netCDF4.Variable'>
float64 time(time)
    long_name: Time
    delta_t: 0000-01-00 00:00:00
    avg_period: 0000-01-00 00:00:00
    prev_avg_period: 0000-00-01 00:00:00
    standard_name: time
    axis: T
    units: hours since 1800-01-01 00:00:0.0
    actual_range: [ 1297320.  1916040.]
unlimited dimensions: time
current shape = (848,)
filling on, default _FillValue of 9.969209968386869e+36 used

我们可以看到时间的属性有很多参数，其中最重要得是units，之所以说它最重要，是因为你看看下面

In [15]:
time[:50]

array([ 1297320.,  1298064.,  1298760.,  1299504.,  1300224.,  1300968.,
        1301688.,  1302432.,  1303176.,  1303896.,  1304640.,  1305360.,
        1306104.,  1306848.,  1307520.,  1308264.,  1308984.,  1309728.,
        1310448.,  1311192.,  1311936.,  1312656.,  1313400.,  1314120.,
        1314864.,  1315608.,  1316280.,  1317024.,  1317744.,  1318488.,
        1319208.,  1319952.,  1320696.,  1321416.,  1322160.,  1322880.,
        1323624.,  1324368.,  1325040.,  1325784.,  1326504.,  1327248.,
        1327968.,  1328712.,  1329456.,  1330176.,  1330920.,  1331640.,
        1332384.,  1333128.])

我显示了时间维的前50个数据，可以看到这里时间的存储方式与我们日常的时间表示方式是不同的，那么这些数字究竟是什么意思呢？

这就要从units中找答案了，我们从上面的属性可以知道，该时间的units为**hours since 1800-01-01 00:00:0.0**   
它的意思是说，这些数字是从1800年1月1日00:00:00作为起始时间以小时作为计数单位累加的结果   
说得通俗点，就是每个时刻距离1800年1月1日00:00:00过去了多少个小时，这个小时数就是该时刻的值   
举例来说1297320的意思是从1800年1月1日00:00:00开始计时的第1297320小时   
简单计算一下

In [16]:
days = 1297320/24
days

54055.0

1297320小时换算过来一共是54055天

In [17]:
years = days / 365
years

148.0958904109589

54055天换算过来是148.096年

In [18]:
1800+years

1948.0958904109589

也就是公历大概1948年1月左右

你看这个计算是不是感觉太繁琐了，难道每个时间都要这样算吗？

显然不是

netCDF4模块包为我们提供了一个换算的函数，以刚才的1297320为例

In [19]:
nc.num2date(1297320,'hours since 1800-01-01 00:00:0.0')

datetime.datetime(1948, 1, 1, 0, 0)

看到没，我们调用num2date函数，传入数据和单位，它就自动生成了准确的时间对象，也就是1948年1月1日00:00

通常情况下，我们的使用习惯是这样的

In [20]:
times = nc.num2date(time[:],time.units)
times[:12]

array([datetime.datetime(1948, 1, 1, 0, 0),
       datetime.datetime(1948, 2, 1, 0, 0),
       datetime.datetime(1948, 3, 1, 0, 0),
       datetime.datetime(1948, 4, 1, 0, 0),
       datetime.datetime(1948, 5, 1, 0, 0),
       datetime.datetime(1948, 6, 1, 0, 0),
       datetime.datetime(1948, 7, 1, 0, 0),
       datetime.datetime(1948, 8, 1, 0, 0),
       datetime.datetime(1948, 9, 1, 0, 0),
       datetime.datetime(1948, 10, 1, 0, 0),
       datetime.datetime(1948, 11, 1, 0, 0),
       datetime.datetime(1948, 12, 1, 0, 0)], dtype=object)

生成的times是一个numpy数组，数组中嵌套的是datetime对象，至于什么是datetime，这就是另一个问题

至此，我们就把netCDF4模块最基础的读取内容演示完了。